In [ ]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))

print(sys.path[-1])  
import os
print(os.getcwd())

C:\RAG_COMMANDER\src
c:\RAG_COMMANDER\src\agents\main


In [ ]:
%%writefile main_agent.py
from langchain_openai import ChatOpenAI
from langgraph.graph.state import Command, Literal
from agents.state.start_state import StartConfirmation, StartInput
from agents.state.main_state import MainState
from utils.llm import LLMProfile
from utils.util import get_today_str
from langchain_core.messages import HumanMessage, get_buffer_string, AIMessage
from langgraph.graph import StateGraph, START, END
from prompts import PromptManager, PromptType
from agents.analysis.analysis_graph import analysis_graph
from agents.jung_min_jae.jung_min_jae_agent import report_graph
from copy import deepcopy

start_llm = LLMProfile.chat_bot_llm()
messages_key = MainState.KEY.messages
start_input_key = MainState.KEY.start_input
analysis_outputs_key = MainState.KEY.analysis_outputs
status_key = MainState.KEY.status


def start_confirmation(
    state: MainState,
) -> Command[Literal["start", "__end__"]]:

    parser_llm = start_llm.with_structured_output(StartConfirmation)
    messages_str = get_buffer_string(messages=state[messages_key])

    prompt = PromptManager(PromptType.MAIN_START_CONFIRMATION).get_prompt(
        messages=messages_str
    )
    response: StartConfirmation = parser_llm.invoke([HumanMessage(content=prompt)])

    if response.confirm == False:
        return Command(
            goto=END, update={messages_key: [AIMessage(content=response.question)]}
        )
    else:
        return Command(
            goto="start",
            update={messages_key: [AIMessage(content=response.verification)]},
        )


def start(state: MainState) -> MainState:
    parser_model = start_llm.with_structured_output(StartInput)
    prompt = PromptManager(PromptType.MAIN_START).get_prompt(
        messages=get_buffer_string(state[messages_key]), date=get_today_str()
    )
    response: StartInput = parser_model.invoke([HumanMessage(content=prompt)])
    return {start_input_key: response.model_dump(), status_key: "ANALYSIS"}


def analysis_graph_node(state: MainState) -> MainState:
    result = analysis_graph.invoke({"start_input": deepcopy(state[start_input_key])})
    return {
        "analysis_outputs": result.get("analysis_outputs", {}),
        status_key: "JUNG_MIN_JAE"
    }


def jung_min_jae_graph(state: MainState) -> MainState:
    result = report_graph.invoke({"start_input": deepcopy(state[start_input_key]),
                                  "analysis_outputs": deepcopy(state[analysis_outputs_key]),
                                  "segment":1
                                  })
    return {
        "final_report": result["final_report"],
        status_key:"RENDERING"
    }


graph_builder = StateGraph(MainState)

start_confirmation_key = "start_confirmation"
start_key = "start"
analysis_graph_key = "analysis_graph"
jung_min_jae_key = "jung_min_jae_graph"

graph_builder.add_node(start_confirmation_key, start_confirmation)
graph_builder.add_node(start_key, start)
graph_builder.add_node(analysis_graph_key, analysis_graph_node)
graph_builder.add_node(jung_min_jae_key, jung_min_jae_graph)

graph_builder.add_edge(START, start_confirmation_key)
graph_builder.add_edge(start_key, analysis_graph_key)
graph_builder.add_edge(analysis_graph_key, jung_min_jae_key)
graph_builder.add_edge(analysis_graph_key, END)


Overwriting main_agent.py


In [ ]:
from main_agent import graph_builder
graph = graph_builder.compile()
graph

c:\PythonProject\RAG_COMMANDER\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'content' from 'langchain_core.messages' (c:\PythonProject\RAG_COMMANDER\.venv\Lib\site-packages\langchain_core\messages\__init__.py)

In [ ]:
from utils.format_message import format_message
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import InMemorySaver
from agents.state.main_state import MainState
messages_key = MainState.KEY.messages
checkpointer = InMemorySaver()
graph = graph_builder.compile(checkpointer = checkpointer)

thread = {"configurable": {"thread_id":"1"}}
result = graph.invoke(
    {
        messages_key : [HumanMessage(content = "경기도 분당구 정자동 백현로 206 근처를 분석하고 싶고 규모는 대단지, 세대수는 1000세대 정도 생각합니다. ")]
    },
    config = thread
)

format_message(result[messages_key])

In [5]:
result

{'messages': [HumanMessage(content='경기도 분당구 정자동 백현로 206 근처를 분석하고 싶고 규모는 대단지, 세대수는 1000세대 정도 생각합니다. ', additional_kwargs={}, response_metadata={}, id='75c6a32f-0ce5-4faf-950c-c42fb3179b85'),
  AIMessage(content='제공해주신 정보(경기도 분당구 정자동 백현로 206 근처, 대단지, 약 1,000세대 규모)를 바탕으로 보고서 작성을 시작하겠습니다. 요청하신 위치와 단지 규모, 세대수가 명확히 확인되었습니다. 이제 부동산 대행사 사내용 보고서를 전문적으로 작성하겠습니다.', additional_kwargs={}, response_metadata={}, id='024bc044-89d5-4c6e-a817-67b2de80b43c')],
 'start_input': {'target_area': '경기도 분당구 정자동 백현로 206 근처',
  'scale': '대단지',
  'total_units': 1000,
  'units_by_type': None,
  'brand': None,
  'orientation': None,
  'parking_ratio': None,
  'terrain_condition': None,
  'gross_area': None,
  'floor_area_ratio_range': None,
  'building_coverage_ratio_range': None},
 'analysis_outputs': {'economic_insight': '분석 기준\n- 대상지: 경기도 성남시 분당구 정자동 백현로 206 인근\n- 기준일: 2025-10-27 (Asia/Seoul, ISO8601)\n\n1) 금리/신용여건\n- 한국은행 기준금리\n  - 최근 결정 및 수준: 2.50% 동결(2025-10-23). 앞선 회의들도 2.50% 동결(2025-08-28, 2025-07-10). 2.50%

In [6]:
print(result['final_report'])

본 보고서는 대한민국 부동산 마케팅 협회 리서치센터가 2025-10-27 기준으로 작성한 공식 자료입니다. 대상지는 경기도 성남시 분당구 정자동 백현로 206 인근으로, 대단지(약 1,000세대) 신규 분양 프로젝트의 분양가·분양성 판단을 위해 경제·입지·주변시장 데이터를 교차 검증했습니다.

Executive Summary — BLUF
- 권장 분양가 밴드: 5,800~6,050만원/3.3㎡(전용 84㎡ 기준 총액 약 14.9~15.6억), 대단지·브랜드·커뮤니티·역세권이 명확히 입증될 경우 상단 6,200만원/3.3㎡ 시도 여지. 6,300만원/3.3㎡ 초과는 구축 중위 대비 +10% 이탈로 미계약 리스크가 유의미하게 상승. [ref: aptndm.com(국토부 실거래 2차 집계), 2025-10; datainKorea, 2025-09]
- 근거: 반경 1km 유사 연식(1994~96년)·유사 평형(전용 84±5㎡) 실거래 중위 5,765만원/3.3㎡, 인근 준신축(2021년) 체감 매매 6,020만원/3.3㎡ 밴드 확인. 신축 분양가는 “구축 상단~준신축 하단 사이” 포지셔닝이 흡수력에 유리. [ref: aptndm.com, 2025-10]
- 거시·정책 환경: 기준금리 2.50%로 12개월 –75bp 하락(2024-10 약 3.25% → 2025-10 2.50%)하며 체감금리 완화가 진행되나, 수도권 대출총량·고가구간 한도(6·27/10·15 보도) 강화로 고가 세대 잔금·중도금 조달은 제약. 결과적으로 분양가 수용여력 상방은 “완만”, 흡수력은 “보통” 축. [ref: TradingEconomics, 2025-10; Bank of Korea(EIEC) 참고자료, 2025-08; KBS, 2025-05; 조선일보, 2025-10]
- 핵심 리스크: 대출총량/DSR 강화, 분양가상한제 적용 여부 미확인, 정밀 입지(역·학교·편의) 거리 데이터 미확보, 반경 3~5km 동시 분양/정비사업 경쟁, >85㎡ 고가 티켓 단기 체류 가능성. [ref: 조선일보, 